In [1]:
import porespy as ps
import numpy as np

from matplotlib.animation import FuncAnimation, PillowWriter
from skimage import measure
from scipy.ndimage import distance_transform_edt
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
from skimage import measure, morphology
from scipy import ndimage
from scipy.ndimage import distance_transform_edt, label
import time
from matplotlib.animation import FuncAnimation, PillowWriter
import os
from scipy.stats import lognorm
from scipy.signal import find_peaks, savgol_filter
from scipy.interpolate import interp1d
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de visualización
try:
    ps.visualization.set_mpl_style()
except:
    plt.style.use('default')

# Medio Poroso

In [2]:
def create_heterogeneous_tight_rock(shape, porosity_target=0.10, voxel_size_um=0.05, seed=None):
    """
    Genera un medio poroso heterogéneo optimizado para rocas tight.
    
    MÉTODO DE RED CON NÚMEROS DE COORDINACIÓN: Genera poros como nodos conectados
    con números de coordinación específicos para garantizar percolación
    """
    
    print(f"="*60)
    print(f"GENERANDO MEDIO POROSO HETEROGÉNEO OPTIMIZADO")
    print(f"="*60)
    print(f"Dimensiones: {shape}")
    print(f"Porosidad objetivo: {porosity_target:.1%}")
    print(f"Tamaño de voxel: {voxel_size_um:.2f} μm")
    
    if seed is not None:
        np.random.seed(seed)
    
    # PASO 1: Definir distribución jerárquica de tamaños (mismas variables)
    large_pore_size_um = 0.8  
    n_large_pores = max(5, int(porosity_target * 0.1 * np.prod(shape) / 1000))
    
    medium_pore_size_um = 0.4  
    n_medium_pores = max(20, int(porosity_target * 0.3 * np.prod(shape) / 500))
    
    small_pore_size_um = 0.2  
    n_small_pores = max(100, int(porosity_target * 0.6 * np.prod(shape) / 200))
    
    print(f"\nDistribución jerárquica de poros:")
    print(f"• Grandes ({large_pore_size_um:.1f} μm): {n_large_pores} poros")
    print(f"• Medianos ({medium_pore_size_um:.1f} μm): {n_medium_pores} poros")
    print(f"• Pequeños ({small_pore_size_um:.1f} μm): {n_small_pores} poros")
    
    # Inicializar medio
    medium = np.zeros(shape, dtype=bool)
    
    # PASO 2: CREAR RED DE POROS CON NÚMEROS DE COORDINACIÓN
    print(f"\nCreando red de poros con números de coordinación controlados...")
    
    # Definir números de coordinación objetivo para cada tipo de poro
    # Poros grandes: alta conectividad (4-6 conexiones)
    # Poros medianos: conectividad media (3-4 conexiones)  
    # Poros pequeños: baja conectividad (2-3 conexiones)
    
    total_pores = n_large_pores + n_medium_pores + n_small_pores
    
    # Generar posiciones de nodos (centros de poros)
    pore_centers = []
    pore_sizes = []
    pore_types = []  # 0: pequeño, 1: mediano, 2: grande
    target_coordination = []
    
    # Función para generar posiciones con mínima distancia
    def generate_positions(n_pores, min_distance, pore_size, pore_type, coord_range):
        positions = []
        for _ in range(n_pores):
            attempts = 0
            while attempts < 100:
                pos = np.array([
                    np.random.randint(10, shape[0]-10),
                    np.random.randint(10, shape[1]-10),
                    np.random.randint(10, shape[2]-10)
                ])
                
                # Verificar distancia mínima con poros existentes
                valid = True
                for existing_pos in pore_centers:
                    if np.linalg.norm(pos - existing_pos) < min_distance:
                        valid = False
                        break
                
                if valid:
                    positions.append(pos)
                    pore_centers.append(pos)
                    pore_sizes.append(pore_size)
                    pore_types.append(pore_type)
                    target_coordination.append(np.random.randint(coord_range[0], coord_range[1]+1))
                    break
                
                attempts += 1
        
        return positions
    
    # Generar poros grandes primero (serán los nodos principales)
    print(f"  Generando {n_large_pores} nodos de poros grandes...")
    large_positions = generate_positions(
        n_large_pores, 
        large_pore_size_um / voxel_size_um * 2,
        large_pore_size_um,
        2,  # tipo grande
        (4, 6)  # coordinación 4-6
    )
    
    # Generar poros medianos
    print(f"  Generando {n_medium_pores} nodos de poros medianos...")
    medium_positions = generate_positions(
        n_medium_pores,
        medium_pore_size_um / voxel_size_um * 1.5,
        medium_pore_size_um,
        1,  # tipo mediano
        (3, 4)  # coordinación 3-4
    )
    
    # Generar poros pequeños
    print(f"  Generando {n_small_pores} nodos de poros pequeños...")
    small_positions = generate_positions(
        n_small_pores,
        small_pore_size_um / voxel_size_um * 1.2,
        small_pore_size_um,
        0,  # tipo pequeño
        (2, 3)  # coordinación 2-3
    )
    
    pore_centers = np.array(pore_centers)
    
    # PASO 3: CONECTAR POROS BASÁNDOSE EN NÚMEROS DE COORDINACIÓN
    print(f"\nConectando poros según números de coordinación...")
    
    # Calcular matriz de distancias
    from scipy.spatial import distance_matrix
    dist_matrix = distance_matrix(pore_centers, pore_centers)
    
    # Inicializar lista de conexiones
    connections = [[] for _ in range(len(pore_centers))]
    coordination_numbers = [0] * len(pore_centers)
    
    # Función para encontrar vecinos potenciales
    def find_potential_neighbors(idx, max_distance):
        distances = dist_matrix[idx]
        neighbors = []
        for j in range(len(pore_centers)):
            if j != idx and distances[j] < max_distance:
                neighbors.append((j, distances[j]))
        return sorted(neighbors, key=lambda x: x[1])
    
    # Conectar poros garantizando percolación en los tres ejes
    # Función para crear camino de percolación usando Dijkstra
    def create_percolation_path(inlet_condition, outlet_condition):
        inlet_pores = []
        outlet_pores = []
        
        for i, center in enumerate(pore_centers):
            if inlet_condition(center):
                inlet_pores.append(i)
            elif outlet_condition(center):
                outlet_pores.append(i)
        
        if inlet_pores and outlet_pores:
            # Encontrar camino más corto entre inlet y outlet
            start_idx = inlet_pores[np.random.randint(len(inlet_pores))]
            end_idx = outlet_pores[np.random.randint(len(outlet_pores))]
            
            # Dijkstra simplificado para encontrar camino
            visited = [False] * len(pore_centers)
            distances = [float('inf')] * len(pore_centers)
            previous = [None] * len(pore_centers)
            distances[start_idx] = 0
            
            for _ in range(len(pore_centers)):
                # Encontrar nodo no visitado con menor distancia
                min_dist = float('inf')
                min_idx = -1
                for i in range(len(pore_centers)):
                    if not visited[i] and distances[i] < min_dist:
                        min_dist = distances[i]
                        min_idx = i
                
                if min_idx == -1:
                    break
                    
                visited[min_idx] = True
                
                # Actualizar distancias de vecinos
                neighbors = find_potential_neighbors(min_idx, shape[0]/2)
                for neighbor_idx, dist in neighbors:
                    if not visited[neighbor_idx]:
                        new_dist = distances[min_idx] + dist
                        if new_dist < distances[neighbor_idx]:
                            distances[neighbor_idx] = new_dist
                            previous[neighbor_idx] = min_idx
            
            # Reconstruir camino
            path = []
            current = end_idx
            while current is not None:
                path.append(current)
                current = previous[current]
            path.reverse()
            
            # Conectar nodos en el camino
            for i in range(len(path)-1):
                if path[i+1] not in connections[path[i]]:
                    connections[path[i]].append(path[i+1])
                    connections[path[i+1]].append(path[i])
                    coordination_numbers[path[i]] += 1
                    coordination_numbers[path[i+1]] += 1
    
    # Crear caminos de percolación en los tres ejes
    # Eje Z (dirección de flujo principal)
    create_percolation_path(
        lambda c: c[2] < 10,  # inlet en z=0
        lambda c: c[2] > shape[2] - 10  # outlet en z=max
    )
    
    # Eje X
    create_percolation_path(
        lambda c: c[0] < 10,  # inlet en x=0
        lambda c: c[0] > shape[0] - 10  # outlet en x=max
    )
    
    # Eje Y
    create_percolation_path(
        lambda c: c[1] < 10,  # inlet en y=0
        lambda c: c[1] > shape[1] - 10  # outlet en y=max
    )
    
    # Completar conexiones basándose en números de coordinación objetivo
    max_connection_distance = shape[0] / 3
    
    for i in range(len(pore_centers)):
        current_coordination = coordination_numbers[i]
        target_coord = target_coordination[i]
        
        if current_coordination < target_coord:
            neighbors = find_potential_neighbors(i, max_connection_distance)
            
            for neighbor_idx, _ in neighbors:
                if current_coordination >= target_coord:
                    break
                    
                neighbor_current = coordination_numbers[neighbor_idx]
                neighbor_target = target_coordination[neighbor_idx]
                
                # Conectar si ambos necesitan más conexiones
                if (neighbor_current < neighbor_target and 
                    neighbor_idx not in connections[i]):
                    
                    connections[i].append(neighbor_idx)
                    connections[neighbor_idx].append(i)
                    coordination_numbers[i] += 1
                    coordination_numbers[neighbor_idx] += 1
                    current_coordination += 1
    
    # PASO 4: CREAR GEOMETRÍA DE POROS Y GARGANTAS
    print(f"\nCreando geometría de poros y gargantas...")
    
    # Coordenadas para esferas
    z_coords, y_coords, x_coords = np.mgrid[0:shape[0], 0:shape[1], 0:shape[2]]
    
    # Función auxiliar para colocar esferas
    def place_sphere(center, radius_um):
        radius_vox = radius_um / (2 * voxel_size_um)
        cz, cy, cx = center
        
        aspect_ratio = np.random.uniform(0.7, 1.3, size=3)
        
        sphere_mask = (
            ((z_coords - cz) / (radius_vox * aspect_ratio[0]))**2 + 
            ((y_coords - cy) / (radius_vox * aspect_ratio[1]))**2 + 
            ((x_coords - cx) / (radius_vox * aspect_ratio[2]))**2
        ) <= 1.0
        
        return sphere_mask
    
    # Crear poros
    for i, center in enumerate(pore_centers):
        size_variation = np.random.uniform(0.8, 1.2)
        medium |= place_sphere(center, pore_sizes[i] * size_variation)
    
    # Crear gargantas (conexiones entre poros)
    channel_radius = int(small_pore_size_um / (2 * voxel_size_um))
    
    for i in range(len(pore_centers)):
        for j in connections[i]:
            if j > i:  # Evitar duplicados
                start = pore_centers[i]
                end = pore_centers[j]
                
                # Radio de garganta basado en los poros que conecta
                throat_radius = min(pore_sizes[i], pore_sizes[j]) * 0.3 / voxel_size_um
                throat_radius = max(1, int(throat_radius))
                
                # Crear cilindro conectando los poros
                n_points = int(np.linalg.norm(end - start)) + 10
                
                for t in np.linspace(0, 1, n_points):
                    point = start + t * (end - start)
                    z, y, x = int(point[0]), int(point[1]), int(point[2])
                    
                    # Crear sección circular
                    for dz in range(-throat_radius, throat_radius+1):
                        for dy in range(-throat_radius, throat_radius+1):
                            for dx in range(-throat_radius, throat_radius+1):
                                if dz**2 + dy**2 + dx**2 <= throat_radius**2:
                                    zz, yy, xx = z+dz, y+dy, x+dx
                                    if 0 <= zz < shape[0] and 0 <= yy < shape[1] and 0 <= xx < shape[2]:
                                        medium[zz, yy, xx] = True
    
    # Imprimir estadísticas de coordinación
    avg_coordination = np.mean(coordination_numbers)
    print(f"\nEstadísticas de números de coordinación:")
    print(f"• Coordinación promedio: {avg_coordination:.2f}")
    print(f"• Rango: {min(coordination_numbers)} - {max(coordination_numbers)}")
    
    base_porosity = np.sum(medium) / medium.size
    print(f"Porosidad base de red: {base_porosity:.2f}")
    
    # PASO 5: Ajuste final de porosidad (igual que original)
    current_porosity = np.sum(medium) / medium.size
    print(f"\nPorosidad después de colocación: {current_porosity:.2f}")
    
    if abs(current_porosity - porosity_target) > 0.01:
        print(f"Ajustando porosidad...")
        
        if current_porosity > porosity_target:
            iterations = 1 if current_porosity < porosity_target * 1.2 else 2
            struct = ndimage.generate_binary_structure(3, 1)
            medium = ndimage.binary_erosion(medium, structure=struct, iterations=iterations)
        elif current_porosity < porosity_target * 0.9:
            struct = ndimage.generate_binary_structure(3, 1)
            medium = ndimage.binary_dilation(medium, structure=struct, iterations=1)
    
    final_porosity = np.sum(medium) / medium.size
    
    # Verificar conectividad en los tres ejes
    print(f"\nVerificando conectividad en los tres ejes:")
    
    # Conectividad en eje X
    inlet_x = np.zeros(shape, dtype=bool)
    inlet_x[:5, :, :] = True
    connected_x = ps.filters.trim_disconnected_blobs(im=medium, inlets=inlet_x)
    connectivity_x = np.sum(connected_x) / np.sum(medium) if np.sum(medium) > 0 else 0
    
    # Conectividad en eje Y
    inlet_y = np.zeros(shape, dtype=bool)
    inlet_y[:, :5, :] = True
    connected_y = ps.filters.trim_disconnected_blobs(im=medium, inlets=inlet_y)
    connectivity_y = np.sum(connected_y) / np.sum(medium) if np.sum(medium) > 0 else 0
    
    # Conectividad en eje Z
    inlet_z = np.zeros(shape, dtype=bool)
    inlet_z[:, :, :5] = True
    connected_z = ps.filters.trim_disconnected_blobs(im=medium, inlets=inlet_z)
    connectivity_z = np.sum(connected_z) / np.sum(medium) if np.sum(medium) > 0 else 0
    
    print(f"• Conectividad eje X: {connectivity_x:.2f}")
    print(f"• Conectividad eje Y: {connectivity_y:.2f}")
    print(f"• Conectividad eje Z: {connectivity_z:.2f}")
    
    # Calcular heterogeneidad real
    dt_final = distance_transform_edt(medium)
    pore_radii = dt_final[medium] * voxel_size_um
    heterogeneity = np.std(pore_radii) / np.mean(pore_radii) if len(pore_radii) > 0 else 0
    
    print(f"\nResultados finales:")
    print(f"• Porosidad final: {final_porosity:.2f}")
    print(f"• Coeficiente de heterogeneidad: {heterogeneity:.2f}")
    
    domain_size_um = [dim * voxel_size_um for dim in shape]
    
    return {
        'medium': medium,
        'shape': shape,
        'voxel_size_um': voxel_size_um,
        'domain_size_um': domain_size_um,
        'porosity_target': porosity_target,
        'porosity_actual': final_porosity,
        'connectivity': {
            'x': connectivity_x,
            'y': connectivity_y,
            'z': connectivity_z
        },
        'heterogeneity': heterogeneity,
        'pore_scales': {
            'large': large_pore_size_um,
            'medium': medium_pore_size_um,
            'small': small_pore_size_um
        }
    }


# Funciones Euler-Poincaré Caracteristico

# Análisis EPC


In [3]:
# ========== FUNCIONES DE ANÁLISIS EPC CORREGIDAS ==========

def calculate_euler_characteristic_robust(binary_image):
    """
    Método robusto y corregido para calcular la característica de Euler.
    La relación correcta es: χ = β₀ - β₁ + β₂
    """
    
    print("  Calculando EPC (método robusto corregido)...")
    
    # Contar componentes conectados (β₀) - usar 26-conectividad para objeto
    struct_elem_26 = ndimage.generate_binary_structure(3, 3)  # 26-conectividad
    labeled, n_components = ndimage.label(binary_image, structure=struct_elem_26)
    
    # Para cavidades (β₂), usar 6-conectividad en el fondo
    struct_elem_6 = ndimage.generate_binary_structure(3, 1)  # 6-conectividad
    inverted = ~binary_image
    labeled_inv, n_cavities_total = ndimage.label(inverted, structure=struct_elem_6)
    
    # Excluir el componente exterior (el que toca los bordes)
    border_labels = set()
    for label in range(1, n_cavities_total + 1):
        mask = labeled_inv == label
        # Verificar si toca algún borde
        if (np.any(mask[0, :, :]) or np.any(mask[-1, :, :]) or 
            np.any(mask[:, 0, :]) or np.any(mask[:, -1, :]) or
            np.any(mask[:, :, 0]) or np.any(mask[:, :, -1])):
            border_labels.add(label)
    
    n_cavities = n_cavities_total - len(border_labels)
    n_cavities = max(0, n_cavities)
    
    # Intentar usar skimage si está disponible
    try:
        from skimage.measure import euler_number
        # Para 3D, euler_number calcula χ correctamente
        euler_char = euler_number(binary_image, connectivity=3)  # 26-conectividad
        
    except ImportError:
        # Implementación manual mejorada
        # Calcular el número de voxels y conexiones para estimar loops
        n_voxels = np.sum(binary_image)
        
        if n_voxels > 0:
            # Estimar conexiones contando vecinos
            kernel = np.ones((3, 3, 3))
            kernel[1, 1, 1] = 0
            neighbors = ndimage.convolve(binary_image.astype(int), kernel, mode='constant')
            total_connections = np.sum(neighbors[binary_image]) // 2  # Cada conexión se cuenta dos veces
            
            # En un árbol perfecto: n_connections = n_voxels - n_components
            # El exceso indica loops
            expected_tree_connections = max(0, n_voxels - n_components)
            excess_connections = max(0, total_connections - expected_tree_connections)
            
            # Estimar loops de manera más conservadora
            # Factor de ajuste basado en la densidad del medio
            density = n_voxels / binary_image.size
            adjustment_factor = 1.0 + density  # Mayor densidad = más loops potenciales
            
            n_loops = int(excess_connections / (26 * adjustment_factor))
            
            # Para medios muy densos y con un solo componente, hay muchos loops
            if n_components == 1 and density > 0.05:
                # Estimación adicional basada en el volumen
                n_loops += int(n_voxels * density * 0.001)
            
        else:
            n_loops = 0
        
        # Calcular χ usando la fórmula correcta
        euler_char = n_components - n_loops + n_cavities
    
    # Calcular β₁ correctamente
    beta_1 = n_components + n_cavities - euler_char
    
    # Verificar consistencia
    if beta_1 < 0:
        print("    ⚠️ Advertencia: β₁ negativo, ajustando...")
        beta_1 = max(0, n_components - 1)  # Como mínimo, algunos loops
        euler_char = n_components - beta_1 + n_cavities
    
    volume = np.sum(binary_image)
    euler_density = euler_char / volume if volume > 0 else 0
    
    print(f"    Componentes (β₀): {n_components}")
    print(f"    Túneles/Loops (β₁): {beta_1}")
    print(f"    Cavidades (β₂): {n_cavities}")
    print(f"    χ = β₀ - β₁ + β₂ = {n_components} - {beta_1} + {n_cavities} = {euler_char}")
    print(f"    χ/V: {euler_density:.6f}")
    
    # Verificación de rango razonable
    if abs(euler_char) > 1000:
        print(f"    ⚠️ χ fuera de rango esperado. Verificar cálculo.")
    elif euler_char < 0:
        print(f"    ✓ χ negativa indica alta conectividad (muchos loops)")
    elif euler_char > 0:
        print(f"    ✓ χ positiva indica baja conectividad (muchos componentes aislados)")
    else:
        print(f"    ✓ χ ≈ 0 cerca del umbral de percolación")
    
    return euler_char, euler_density, n_components

# Reemplazar la función errónea con la corregida
calculate_euler_characteristic_local_config = calculate_euler_characteristic_robust





def calculate_connectivity_function(medium, voxel_size_um=0.05, n_points=30):
    """
    Calcula la función de conectividad χ(d) variando el diámetro de poro.
    Versión corregida con el cálculo apropiado de EPC.
    """
    
    print("\n" + "="*60)
    print("CALCULANDO FUNCIÓN DE CONECTIVIDAD EPC (VERSIÓN CORREGIDA)")
    print("="*60)
    
    # Distance transform del volumen 3D completo
    dt = distance_transform_edt(medium)
    max_radius_vox = np.max(dt)
    max_radius_um = max_radius_vox * voxel_size_um
    
    print(f"Radio máximo en el medio 3D: {max_radius_um:.2f} μm")
    print(f"Volumen total del medio: {medium.shape}")
    print(f"Voxels porosos: {np.sum(medium):,}")
    
    # Calcular EPC inicial del medio original
    euler_initial, euler_dens_initial, beta_0_initial = calculate_euler_characteristic_robust(medium)
    print(f"\nEPC inicial del medio: χ = {euler_initial} (β₀ = {beta_0_initial})")
    
    if euler_initial > 0:
        print("⚠️ ADVERTENCIA: χ inicial positiva - esto puede indicar:")
        print("   - Medio poco conectado")
        print("   - Muchos componentes aislados")
        print("   - Estructura dominada por poros aislados")
    else:
        print("✓ χ inicial negativa - indica alta conectividad del medio")
        print(f"  Magnitud |χ| = {abs(euler_initial)} sugiere alta conectividad")
    
    # Radios a evaluar (incluir 0 para el estado inicial)
    radii_um = np.concatenate([[0], np.linspace(0.01, max_radius_um * 0.9, n_points-1)])
    
    # Arrays para resultados
    euler_chars = []
    euler_densities = []
    porosities = []
    n_components = []
    
    print("\nCalculando EPC para diferentes radios (morphological opening):")
    print("Radio (μm) | χ | β₀ | Porosidad | Comentario")
    print("-" * 60)
    
    # Primero, el estado inicial (radio = 0)
    euler, euler_dens, beta_0 = euler_initial, euler_dens_initial, beta_0_initial
    euler_chars.append(euler)
    euler_densities.append(euler_dens)
    porosities.append(np.sum(medium) / medium.size)
    n_components.append(beta_0)
    
    print(f"{0:9.3f} | {euler:6d} | {beta_0:4d} | {porosities[0]:9.4f} | Estado inicial")
    
    # Aplicar morphological opening para los demás radios
    opened_images = [medium.copy()]  # Incluir imagen original
    
    for i, radius in enumerate(radii_um[1:], 1):
        # Aplicar morphological opening
        if radius < voxel_size_um:
            opened = medium.copy()
        else:
            radius_vox = int(radius / voxel_size_um)
            if radius_vox < 1:
                opened = medium.copy()
            else:
                # Crear elemento estructural esférico
                struct = morphology.ball(radius_vox)
                
                # Aplicar opening: erosión seguida de dilatación
                opened = morphology.binary_opening(medium, struct)
        
        opened_images.append(opened)
        
        # Calcular EPC del volumen abierto
        if np.sum(opened) > 0:
            euler, euler_dens, beta_0 = calculate_euler_characteristic_robust(opened)
            porosity = np.sum(opened) / opened.size
        else:
            euler, euler_dens, beta_0 = 0, 0, 0
            porosity = 0
        
        euler_chars.append(euler)
        euler_densities.append(euler_dens)
        porosities.append(porosity)
        n_components.append(beta_0)
        
        # Imprimir progreso para radios clave
        if i == 1 or i % 5 == 0 or i == len(radii_um)-1:
            comment = ""
            if euler > 0 and euler_chars[i-1] <= 0:
                comment = "← Transición a baja conectividad"
            elif beta_0 > n_components[i-1] * 2:
                comment = "← Fragmentación significativa"
            
            print(f"{radius:9.2f} | {euler:6d} | {beta_0:4d} | {porosity:9.4f} | {comment}")
    
    # Convertir a arrays
    radii_um = np.array(radii_um)
    euler_chars = np.array(euler_chars)
    euler_densities = np.array(euler_densities)
    porosities = np.array(porosities)
    n_components = np.array(n_components)
    
    # Análisis de la evolución de χ
    print("\nAnálisis de la evolución de χ:")
    
    # Verificar monotonía
    dchi = np.diff(euler_chars)
    if np.all(dchi >= 0):
        print("✓ χ aumenta monotónicamente (comportamiento esperado)")
    else:
        n_decreases = np.sum(dchi < 0)
        print(f"⚠ χ no es estrictamente monotónica ({n_decreases} decrementos)")
    
    # Encontrar radio crítico donde χ cruza cero
    critical_radius = None
    sign_changes = np.where(np.diff(np.sign(euler_chars)))[0]
    
    if len(sign_changes) > 0:
        # Buscar el cruce de negativo a positivo
        for idx in sign_changes:
            if euler_chars[idx] < 0 and euler_chars[idx+1] > 0:
                r1, r2 = radii_um[idx], radii_um[idx+1]
                chi1, chi2 = euler_chars[idx], euler_chars[idx+1]
                
                # Interpolación lineal para encontrar el cruce exacto
                critical_radius = r1 - chi1 * (r2 - r1) / (chi2 - chi1)
                
                print(f"\n✓ Radio crítico encontrado: r_c = {critical_radius:.2f} μm")
                print(f"  Transición: r={r1:.3f} (χ={chi1}) → r={r2:.3f} (χ={chi2})")
                print(f"  En $R_c$: umbral de percolación del medio")
                break
        
        if critical_radius is None:
            # No se encontró cruce de negativo a positivo
            critical_radius = radii_um[sign_changes[0]]
            print(f"\n⚠ Cruce encontrado pero atípico en r = {critical_radius:.2f} μm")
    else:
        # Si no hay cruce, buscar el punto más cercano a cero
        min_idx = np.argmin(np.abs(euler_chars))
        critical_radius = radii_um[min_idx]
        min_chi = euler_chars[min_idx]
        
        if min_chi < 0:
            print(f"\n⚠ No se encontró cruce por cero.")
            print(f"  χ permanece negativa. Mínimo |χ| = {abs(min_chi)} en r = {critical_radius:.3f} μm")
            print(f"  El medio mantiene conectividad incluso para radios grandes")
        else:
            print(f"\n⚠ χ siempre positiva. Mínimo χ = {min_chi} en r = {critical_radius:.3f} μm")
            print(f"  El medio tiene baja conectividad inicial")
    
    # Análisis adicional
    print(f"\nResumen del análisis topológico:")
    print(f"• Rango de χ: [{min(euler_chars)}, {max(euler_chars)}]")
    print(f"• Componentes: {n_components[0]} → {n_components[-1]}")
    print(f"• Reducción de porosidad: {porosities[0]:.1%} → {porosities[-1]:.1%}")
    
    # Clasificar el tipo de medio según el comportamiento de χ
    if euler_chars[0] < -100:
        print(f"• Tipo de medio: Altamente conectado (muchos loops)")
    elif euler_chars[0] < 0:
        print(f"• Tipo de medio: Bien conectado")
    elif euler_chars[0] < 100:
        print(f"• Tipo de medio: Conectividad moderada")
    else:
        print(f"• Tipo de medio: Poco conectado (muchos poros aislados)")
    
    # Verificar rango razonable
    if max(np.abs(euler_chars)) > 1000:
        print(f"\n⚠️ ADVERTENCIA: Valores de χ fuera del rango esperado [-1000, 1000]")
        print(f"   Esto puede indicar un error en el cálculo o un medio muy especial")
    
    return {
        'radii': radii_um,
        'euler_chars': euler_chars,
        'euler_densities': euler_densities,
        'porosities': porosities,
        'n_components': n_components,
        'critical_radius': critical_radius,
        'opened_images': opened_images
    }


def calculate_connectivity_function_enhanced(medium, voxel_size_um=0.05, n_points=30):
    """
    Versión mejorada con detección de cambios abruptos.
    """
    
    # Primero ejecutar el análisis original
    epc_results = calculate_connectivity_function(medium, voxel_size_um, n_points)
    
    # Luego añadir el análisis de cambios abruptos
    critical_radius_physical, abrupt_changes = detect_abrupt_changes_in_connectivity(
        epc_results['radii'],
        epc_results['euler_chars'],
        epc_results['n_components'],
        epc_results['porosities'],
        voxel_size_um
    )
    
    # Añadir resultados al diccionario
    epc_results['critical_radius_physical'] = critical_radius_physical
    epc_results['critical_radius_mathematical'] = epc_results['critical_radius']  # χ = 0
    epc_results['abrupt_changes_analysis'] = abrupt_changes
    
    # Comparar radios críticos
    if critical_radius_physical and epc_results['critical_radius']:
        print(f"\nCOMPARACIÓN DE RADIOS CRÍTICOS:")
        print(f"• Radio crítico físico (cambios abruptos): {critical_radius_physical:.3f} μm")
        print(f"• Radio crítico matemático (χ = 0): {epc_results['critical_radius']:.3f} μm")
        print(f"• Diferencia: {abs(epc_results['critical_radius'] - critical_radius_physical):.3f} μm")
        
        if critical_radius_physical < epc_results['critical_radius']:
            print("✓ El radio crítico físico ocurre antes (comportamiento esperado)")
        else:
            print("⚠️ El radio crítico matemático ocurre antes (revisar heterogeneidad)")
    
    return epc_results

# $R_c$ detection

In [4]:
def detect_abrupt_changes_in_connectivity(radii, euler_chars, n_components, porosities, voxel_size_um):
    """
    Detecta cambios abruptos en la función de conectividad χ(r).
    Busca el punto donde la conectividad se pierde más rápidamente (máximo en dχ/dr).
    """
    
    print("\n" + "="*60)
    print("ANÁLISIS DE CAMBIOS ABRUPTOS EN CONECTIVIDAD")
    print("="*60)
    
    # Preparar datos
    n_points = len(radii)
    
    # 1. Calcular primera derivada de χ
    dchi_dr = np.gradient(euler_chars, radii)
    
    # 2. Suavizar para reducir ruido
    window_length = min(7, n_points if n_points % 2 == 1 else n_points - 1)
    if window_length >= 3:
        dchi_dr_smooth = savgol_filter(dchi_dr, window_length=window_length, polyorder=2)
    else:
        dchi_dr_smooth = dchi_dr
    
    # 3. Calcular segunda derivada para detectar cambios en pendiente
    d2chi_dr2 = np.gradient(dchi_dr_smooth, radii)
    
    # 4. Identificar región de transición (donde χ cruza cero)
    # Encontrar índices donde χ cambia de signo
    sign_changes = np.where(np.diff(np.sign(euler_chars)))[0]
    
    # 5. Buscar máximos en la derivada (pérdida de conectividad)
    # IMPORTANTE: Buscar máximos en dχ/dr, NO en |dχ/dr|
    # Un máximo en dχ/dr indica pérdida rápida de conectividad
    
    # Establecer umbral basado en la distribución de la derivada
    positive_derivs = dchi_dr_smooth[dchi_dr_smooth > 0]
    if len(positive_derivs) > 0:
        threshold = np.percentile(positive_derivs, 75)  # Percentil 75 de valores positivos
    else:
        threshold = 0
    
    # Buscar máximos locales en la derivada
    maxima, max_properties = find_peaks(dchi_dr_smooth, 
                                       height=threshold,
                                       prominence=np.std(dchi_dr_smooth) * 0.5)
    
    # También buscar mínimos para completitud
    minima, min_properties = find_peaks(-dchi_dr_smooth,
                                       prominence=np.std(dchi_dr_smooth) * 0.5)
    
    # 6. Analizar cambios en componentes conectados
    log_components = np.log10(n_components + 1)
    d_log_comp = np.gradient(log_components, radii)
    
    # 7. Detectar candidatos a radio crítico
    critical_radii_candidates = []
    
    # Método 1: Máximo más prominente en dχ/dr (pérdida rápida de conectividad)
    if len(maxima) > 0:
        # Ordenar por altura (magnitud del cambio)
        heights = dchi_dr_smooth[maxima]
        sorted_indices = np.argsort(heights)[::-1]  # Orden descendente
        
        # Tomar el máximo más prominente
        primary_max_idx = maxima[sorted_indices[0]]
        r_primary_max = radii[primary_max_idx]
        
        critical_radii_candidates.append(('Máxima pérdida de conectividad', r_primary_max, primary_max_idx))
        
        print(f"\nMétodo 1 - Máxima pérdida de conectividad (máximo en d$\chi$/dr):")
        print(f"  Radio: {r_primary_max:.3f} μm")
        print(f"  dχ/dr: {dchi_dr_smooth[primary_max_idx]:.2f}")
        print(f"  χ en este punto: {euler_chars[primary_max_idx]}")
        print(f"  Componentes: {n_components[primary_max_idx]}")
        
        # Si hay un segundo máximo cercano, también considerarlo
        if len(maxima) > 1:
            for i in sorted_indices[1:]:
                if abs(maxima[i] - primary_max_idx) <= 5:  # Cercano al primero
                    secondary_max_idx = maxima[i]
                    r_secondary = radii[secondary_max_idx]
                    print(f"\n  Máximo secundario cercano:")
                    print(f"    Radio: {r_secondary:.3f} μm")
                    print(f"    dχ/dr: {dchi_dr_smooth[secondary_max_idx]:.2f}")
                    break
    
    # Método 2: Punto donde χ cruza cero (transición topológica)
    if len(sign_changes) > 0:
        # Tomar el primer cruce de negativo a positivo
        for sc_idx in sign_changes:
            if euler_chars[sc_idx] < 0 and euler_chars[sc_idx + 1] >= 0:
                # Interpolar para encontrar el punto exacto donde χ = 0
                r1, r2 = radii[sc_idx], radii[sc_idx + 1]
                chi1, chi2 = euler_chars[sc_idx], euler_chars[sc_idx + 1]
                r_zero = r1 - chi1 * (r2 - r1) / (chi2 - chi1)
                
                critical_radii_candidates.append(('Transición topológica (χ=0)', r_zero, sc_idx))
                
                print(f"\nMétodo 2 - Transición topológica (χ cruza cero):")
                print(f"  Radio interpolado: {r_zero:.3f} μm")
                print(f"  Entre índices {sc_idx} y {sc_idx + 1}")
                break
    
    # Método 3: Mayor cambio en componentes conectados
    comp_peaks, _ = find_peaks(d_log_comp, prominence=0.1)
    if len(comp_peaks) > 0:
        max_comp_change_idx = comp_peaks[np.argmax(d_log_comp[comp_peaks])]
        r_comp_change = radii[max_comp_change_idx]
        critical_radii_candidates.append(('Fragmentación máxima', r_comp_change, max_comp_change_idx))
        
        print(f"\nMétodo 3 - Fragmentación de componentes:")
        print(f"  Radio: {r_comp_change:.3f} μm")
        print(f"  Componentes: {n_components[max(0, max_comp_change_idx-1)]} → {n_components[max_comp_change_idx]}")
    
    # Método 4: Análisis de curvatura (puntos de inflexión)
    inflection_points = np.where(np.diff(np.sign(d2chi_dr2)))[0]
    if len(inflection_points) > 0:
        # Buscar punto de inflexión cerca de un máximo en dχ/dr
        for ip in inflection_points:
            if ip > 0 and ip < len(dchi_dr_smooth) - 1:
                # Verificar si está cerca de un máximo en la derivada
                if dchi_dr_smooth[ip] > threshold:
                    r_inflection = radii[ip]
                    critical_radii_candidates.append(('Punto de inflexión', r_inflection, ip))
                    
                    print(f"\nMétodo 4 - Punto de inflexión:")
                    print(f"  Radio: {r_inflection:.3f} μm")
                    print(f"  d²χ/dr²: {d2chi_dr2[ip]:.2f}")
                    break
    
    # Método 5: Caída significativa de porosidad
    if len(porosities) > 0 and porosities[0] > 0:
        porosity_ratio = porosities / porosities[0]
        
        # Buscar donde la porosidad cae por debajo del 50%
        porosity_drop_idx = np.where(porosity_ratio < 0.5)[0]
        if len(porosity_drop_idx) > 0:
            idx_50 = porosity_drop_idx[0]
            r_porosity_50 = radii[idx_50]
            critical_radii_candidates.append(('Pérdida 50% porosidad', r_porosity_50, idx_50))
            
            print(f"\nMétodo 5 - Pérdida de porosidad conectada:")
            print(f"  Radio (50% pérdida): {r_porosity_50:.3f} μm")
            print(f"  Porosidad: {porosities[0]:.3f} → {porosities[idx_50]:.3f}")
    
    # Seleccionar el radio crítico físico
    if critical_radii_candidates:
        # Priorizar el método del máximo en dχ/dr si existe
        max_deriv_candidates = [c for c in critical_radii_candidates if 'Máxima pérdida' in c[0]]
        
        if max_deriv_candidates:
            primary_method, primary_radius, primary_idx = max_deriv_candidates[0]
        else:
            # Si no hay máximo claro, usar el candidato más conservador (menor radio)
            critical_radii_candidates.sort(key=lambda x: x[1])
            primary_method, primary_radius, primary_idx = critical_radii_candidates[0]
        
        print(f"\n" + "="*50)
        print(f"RADIO CRÍTICO FÍSICO DETECTADO:")
        print(f"  Método: {primary_method}")
        print(f"  Radio: {primary_radius:.3f} μm")
        print(f"  χ: {euler_chars[min(primary_idx, len(euler_chars)-1)]:.0f}")
        print(f"  β₀: {n_components[min(primary_idx, len(n_components)-1)]}")
        print(f"  Porosidad: {porosities[min(primary_idx, len(porosities)-1)]:.3f}")
        
        # Análisis de confianza
        confidence = "Alta" if len(critical_radii_candidates) >= 3 else "Media" if len(critical_radii_candidates) >= 2 else "Baja"
        print(f"  Confianza: {confidence} ({len(critical_radii_candidates)} métodos coincidentes)")
        print("="*50)
        
        # Preparar resultados detallados
        results = {
            'primary_radius': primary_radius,
            'primary_method': primary_method,
            'primary_idx': primary_idx,
            'all_candidates': critical_radii_candidates,
            'dchi_dr': dchi_dr,
            'dchi_dr_smooth': dchi_dr_smooth,
            'd2chi_dr2': d2chi_dr2,
            'maxima': maxima,
            'minima': minima,
            'max_properties': max_properties,
            'confidence': confidence
        }
        
        return primary_radius, results
    else:
        print("\n⚠️ No se detectaron cambios abruptos significativos")
        return None, None

# EPC STATIC

In [5]:
def create_epc_static_comparison(epc_results):
    """
    Crea una imagen estática mostrando estados clave de la evolución EPC.
    Con formato homogeneizado de fuentes y porosidad real.
    CORREGIDO: Escalas en micrómetros para visualización 3D.
    """
    
    # ============= CONFIGURACIÓN GLOBAL DE ESTILO =============
    # Definir tamaños de fuente consistentes
    FONT_SIZE_MAIN = 20        # Tamaño principal para ejes y etiquetas
    FONT_SIZE_TITLE = 22       # Tamaño para títulos de subplots
    FONT_SIZE_SUPTITLE = 24    # Tamaño para título principal
    FONT_SIZE_SMALL = 16       # Tamaño para anotaciones y textos secundarios
    FONT_SIZE_LEGEND = 18      # Tamaño para leyendas
    
    # Configurar matplotlib para usar sans-serif
    plt.rcParams.update({
        'font.family': 'sans-serif',
        'font.serif': ['Sans-serif'], #'Times New Roman', 'DejaVu Serif', 'Times', 'serif'],
        'font.size': FONT_SIZE_MAIN,
        'axes.labelsize': FONT_SIZE_MAIN,
        'axes.titlesize': FONT_SIZE_TITLE,
        'xtick.labelsize': FONT_SIZE_MAIN,
        'ytick.labelsize': FONT_SIZE_MAIN,
        'legend.fontsize': FONT_SIZE_LEGEND,
        'figure.titlesize': FONT_SIZE_SUPTITLE,
    })
    # =========================================================
    
    # Obtener parámetros del dominio
    voxel_size_um = epc_results.get('voxel_size_um', 0.05)
    domain_size_um = epc_results.get('domain_size_um', None)
    
    # Si no está disponible domain_size_um, calcularlo
    if domain_size_um is None:
        if 'shape' in epc_results:
            shape = epc_results['shape']
        else:
            # Usar la forma de la primera imagen
            shape = epc_results['opened_images'][0].shape
        domain_size_um = [dim * voxel_size_um for dim in shape]
    
    print(f"Creando comparación estática para dominio: {domain_size_um[0]:.1f} × {domain_size_um[1]:.1f} × {domain_size_um[2]:.1f} μm³")
    
    # Seleccionar 4 estados representativos
    n_total = len(epc_results['radii'])
    indices = [0, n_total//3, 2*n_total//3, n_total-1]
    
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(2, 4, height_ratios=[1.5, 1], hspace=0.3, wspace=0.3)
    
    # Fila superior: Vistas 3D
    step = 3
    scale_factor = voxel_size_um * step  # Factor de escala para convertir a μm
    
    for i, idx in enumerate(indices):
        ax = fig.add_subplot(gs[0, i], projection='3d')
        
        radius = epc_results['radii'][idx]
        opened = epc_results['opened_images'][idx]
        opened_vis = opened[::step, ::step, ::step]
        
        # Calcular porosidad REAL (fracción de voxels porosos respecto al volumen total)
        porosity_real = np.sum(opened) / opened.size
        
        if np.sum(opened_vis) > 10:
            try:
                verts, faces, _, _ = measure.marching_cubes(opened_vis.astype(float), level=0.5)
                
                # CORRECCIÓN: Convertir vértices a micrómetros
                verts_um = verts * scale_factor
                
                # Color según conectividad
                if epc_results['euler_densities'][idx] < 0:
                    color = plt.cm.Blues(0.7)
                else:
                    color = plt.cm.Reds(0.7)
                
                ax.plot_trisurf(verts_um[:, 0], verts_um[:, 1], verts_um[:, 2],
                               triangles=faces, color=color, alpha=0.8,
                               edgecolor='none')
            except:
                ax.text(0.5, 0.5, 0.5, 'Sin datos', transform=ax.transAxes, 
                       ha='center', va='center', fontsize=FONT_SIZE_SMALL,
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        else:
            ax.text(0.5, 0.5, 0.5, 'Sin estructura\nconectada', transform=ax.transAxes, 
                   ha='center', va='center', fontsize=FONT_SIZE_SMALL,
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # CORRECCIÓN: Establecer límites en micrómetros
        ax.set_xlim([0, domain_size_um[2]])  # X en μm
        ax.set_ylim([0, domain_size_um[1]])  # Y en μm
        ax.set_zlim([0, domain_size_um[0]])  # Z en μm
        
        # Etiquetas de ejes con formato consistente
        ax.set_xlabel('X (μm)', fontsize=FONT_SIZE_MAIN)
        ax.set_ylabel('Y (μm)', fontsize=FONT_SIZE_MAIN)
        ax.set_zlabel('Z (μm)', fontsize=FONT_SIZE_MAIN, 
                     rotation=0, ha='right')
        
        # Configurar tamaño de ticks
        ax.tick_params(axis='x', labelsize=FONT_SIZE_SMALL)
        ax.tick_params(axis='y', labelsize=FONT_SIZE_SMALL)
        ax.tick_params(axis='z', labelsize=FONT_SIZE_SMALL)
        
        # Configurar número de ticks para evitar amontonamiento
        ax.xaxis.set_major_locator(plt.MaxNLocator(4))
        ax.yaxis.set_major_locator(plt.MaxNLocator(4))
        ax.zaxis.set_major_locator(plt.MaxNLocator(4))
        
        # Título con información (ahora con porosidad real)
        ax.set_title( 
            f'r = {radius:.2f} μm\n'
            f'χ = {epc_results["euler_chars"][idx]:.0f}\n'
            f'$\phi$ = {porosity_real * 100:.1f}%',
            fontsize=FONT_SIZE_MAIN,
            weight='normal'
        )

        ax.view_init(elev=30, azim=45)
        ax.grid(True, alpha=0.3)
    
    # Fila inferior: Función de conectividad completa
    ax_bottom = fig.add_subplot(gs[1, :])
    
    ax_bottom.plot(epc_results['radii'], epc_results['euler_chars'], 
                  'b-', linewidth=2.5)
    ax_bottom.axhline(y=0, color='r', linestyle='--', alpha=0.7, linewidth=1.5)
    
    # Radio crítico topológico
    if 'critical_radius' in epc_results and epc_results['critical_radius'] is not None:
        ax_bottom.axvline(x=epc_results['critical_radius'], color='g', 
                         linestyle='--', linewidth=2,
                         label=f'$R_c$ ($\chi(r) = 0$) = {epc_results["critical_radius"]:.2f} μm')
    
    # Radio crítico físico
    if 'critical_radius_physical' in epc_results and epc_results['critical_radius_physical'] is not None:
        ax_bottom.axvline(x=epc_results['critical_radius_physical'], color='red', 
                         linestyle=':', linewidth=2,
                         label=f'$R_c$ (derivada) = {epc_results["critical_radius_physical"]:.2f} μm')
    
    # Marcar los puntos seleccionados
    for idx in indices:
        ax_bottom.plot(epc_results['radii'][idx], epc_results['euler_chars'][idx], 
                      'ro', markersize=10)
    
    # Sombrear regiones
    ax_bottom.fill_between(epc_results['radii'], 0, epc_results['euler_chars'],
                          where=(epc_results['euler_chars'] < 0),
                          color='blue', alpha=0.1, label='Alta conectividad')
    ax_bottom.fill_between(epc_results['radii'], 0, epc_results['euler_chars'],
                          where=(epc_results['euler_chars'] > 0),
                          color='red', alpha=0.1, label='Baja connectividad')
    
    ax_bottom.set_xlabel('Radio de poro (μm)', fontsize=FONT_SIZE_MAIN)
    
    ax_bottom.set_ylabel(r'$\chi(r)$ (adimensional)', fontsize=FONT_SIZE_MAIN)
    
    #ax_bottom.set_title('Connectivity Function', fontsize=FONT_SIZE_TITLE)
    ax_bottom.grid(True, alpha=0.3)
    ax_bottom.legend(fontsize=FONT_SIZE_LEGEND, loc='best', ncol=2)
    ax_bottom.tick_params(axis='both', which='major', labelsize=FONT_SIZE_MAIN)
    
    # Título principal con información del dominio
    domain_info = f" (Domain: {domain_size_um[0]:.1f} × {domain_size_um[1]:.1f} × {domain_size_um[2]:.1f} μm³)"
    #plt.suptitle('EPC Evolution: Key States' + domain_info, fontsize=FONT_SIZE_SUPTITLE)
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    
    filename = 'epc_evolution_3d_static.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
    print(f"Imagen estática EPC guardada como: {filename}")
    print(f"Dominio visualizado: {domain_size_um[0]:.1f} × {domain_size_um[1]:.1f} × {domain_size_um[2]:.1f} μm³")
    plt.close(fig)

# EPC ANIMATION

In [6]:

def create_animated_epc_evolution(epc_results, fps=2):
    """
    Crea una animación GIF mostrando la evolución de la conectividad con vistas isométricas 3D.
    Panel izquierdo: Vista 3D isométrica del medio poroso completo
    Panel derecho: Función de conectividad dinámica
    Con formato homogeneizado de fuentes y porosidad real.
    CORREGIDO: Escalas en micrómetros para visualización 3D.
    """
    
    # ============= CONFIGURACIÓN GLOBAL DE ESTILO =============
    # Definir tamaños de fuente consistentes
    FONT_SIZE_MAIN = 20        # Tamaño principal para ejes y etiquetas
    FONT_SIZE_TITLE = 22       # Tamaño para títulos de subplots
    FONT_SIZE_SUPTITLE = 24    # Tamaño para título principal
    FONT_SIZE_SMALL = 16       # Tamaño para anotaciones y textos secundarios
    FONT_SIZE_LEGEND = 18      # Tamaño para leyendas
    
    # Configurar matplotlib para usar sans-serif
    plt.rcParams.update({
        'font.family': 'sans-serif',
        'font.serif': ["Sans-serif"],  #'Times New Roman', 'DejaVu Serif', 'Times', 'serif'],
        'font.size': FONT_SIZE_MAIN,
        'axes.labelsize': FONT_SIZE_MAIN,
        'axes.titlesize': FONT_SIZE_TITLE,
        'xtick.labelsize': FONT_SIZE_MAIN,
        'ytick.labelsize': FONT_SIZE_MAIN,
        'legend.fontsize': FONT_SIZE_LEGEND,
        'figure.titlesize': FONT_SIZE_SUPTITLE,
    })
    # =========================================================
    
    print("\nCreando animación 3D de evolución EPC...")
    
    # Obtener parámetros del dominio
    voxel_size_um = epc_results.get('voxel_size_um', 0.05)
    domain_size_um = epc_results.get('domain_size_um', None)
    
    # Si no está disponible domain_size_um, calcularlo
    if domain_size_um is None:
        if 'shape' in epc_results:
            shape = epc_results['shape']
        else:
            # Usar la forma de la primera imagen
            shape = epc_results['opened_images'][0].shape
        domain_size_um = [dim * voxel_size_um for dim in shape]
    
    print(f"Dominio para animación: {domain_size_um[0]:.1f} × {domain_size_um[1]:.1f} × {domain_size_um[2]:.1f} μm³")
    
    n_frames = min(20, len(epc_results['radii']))
    frame_indices = np.linspace(0, len(epc_results['radii'])-1, n_frames, dtype=int)
    
    # Crear figura con proporciones adecuadas
    fig = plt.figure(figsize=(16, 8), facecolor='white')
    
    # Submuestreo para visualización 3D
    step = 3
    scale_factor = voxel_size_um * step  # Factor de escala para convertir a μm
    
    # Pre-calcular límites para consistencia
    euler_min = min(epc_results['euler_chars'])
    euler_max = max(epc_results['euler_chars'])
    
    # Asegurar que el rango incluya cero
    if euler_min > 0:
        euler_min = -abs(euler_max) * 0.1
    if euler_max < 0:
        euler_max = abs(euler_min) * 0.1
    
    print(f"Rango de χ para visualización: [{euler_min:.0f}, {euler_max:.0f}]")
    
    def update(frame_idx):
        fig.clear()
        
        idx = frame_indices[frame_idx]
        radius = epc_results['radii'][idx]
        
        # Crear subplots para cada frame
        ax1 = fig.add_subplot(121, projection='3d')
        ax2 = fig.add_subplot(122)
        
        # Panel 1: Vista isométrica 3D del medio completo
        opened = epc_results['opened_images'][idx]
        opened_vis = opened[::step, ::step, ::step]
        
        # Calcular porosidad REAL (fracción de voxels porosos respecto al volumen total)
        porosity_real = np.sum(opened) / opened.size
        
        n_components = epc_results['n_components'][idx]
        euler_char = epc_results['euler_chars'][idx]
        
        # Crear isosuperficie si hay suficientes voxels
        if np.sum(opened_vis) > 10:
            try:
                verts, faces, _, _ = measure.marching_cubes(opened_vis.astype(float), level=0.5)
                
                # CORRECCIÓN: Convertir vértices a micrómetros
                verts_um = verts * scale_factor
                
                # Color basado en el valor de χ
                if euler_char < 0:
                    color_map = plt.cm.Blues(0.7)  # Azul para alta conectividad
                else:
                    color_map = plt.cm.Reds(0.7)   # Rojo para baja conectividad
                
                # Renderizar superficie con coordenadas en μm
                surf = ax1.plot_trisurf(verts_um[:, 0], verts_um[:, 1], verts_um[:, 2],
                                       triangles=faces, 
                                       color=color_map,
                                       alpha=0.85,
                                       edgecolor='none',
                                       linewidth=0,
                                       shade=True)
                
            except Exception as e:
                # Si falla, mostrar nube de puntos
                coords = np.where(opened_vis)
                if len(coords[0]) > 0:
                    # Limitar número de puntos
                    n_points = min(5000, len(coords[0]))
                    indices = np.random.choice(len(coords[0]), n_points, replace=False)
                    
                    # CORRECCIÓN: Convertir coordenadas a micrómetros
                    x_um = coords[2][indices] * scale_factor
                    y_um = coords[1][indices] * scale_factor
                    z_um = coords[0][indices] * scale_factor
                    
                    ax1.scatter(x_um, y_um, z_um, 
                              c='black', s=0.5, alpha=0.3)
        else:
            ax1.text(0.5, 0.5, 0.5, 'Sin estructura\nconectada', 
                    transform=ax1.transAxes, ha='center', va='center',
                    fontsize=FONT_SIZE_SMALL, 
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # CORRECCIÓN: Configurar ejes 3D con límites en micrómetros
        ax1.set_xlim([0, domain_size_um[2]])  # X en μm
        ax1.set_ylim([0, domain_size_um[1]])  # Y en μm
        ax1.set_zlim([0, domain_size_um[0]])  # Z en μm
        
        ax1.set_xlabel('X (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
        ax1.set_ylabel('Y (μm)', fontsize=FONT_SIZE_MAIN, labelpad=10)
        ax1.set_zlabel('Z (μm)', fontsize=FONT_SIZE_MAIN, 
                      rotation=0, ha='right', labelpad=10)
        
        # Configurar tamaño de ticks
        ax1.tick_params(axis='x', labelsize=FONT_SIZE_SMALL)
        ax1.tick_params(axis='y', labelsize=FONT_SIZE_SMALL)
        ax1.tick_params(axis='z', labelsize=FONT_SIZE_SMALL)
        
        # Configurar número de ticks para evitar amontonamiento
        ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
        ax1.yaxis.set_major_locator(plt.MaxNLocator(5))
        ax1.zaxis.set_major_locator(plt.MaxNLocator(5))
               
        # Título con información (ahora con porosidad real)
        chi_sign = "+" if euler_char >= 0 else ""
        connectivity_state = "Alta conectividad" if euler_char < 0 else "Baja conectividad"
        ax1.set_title(f'\n'
                     f'Radio = {radius:.2f} μm | χ = {chi_sign}{euler_char}\n'
                     f'$\phi$ = {porosity_real * 100:.1f}% | {connectivity_state}', 
                     fontsize=FONT_SIZE_SMALL, pad=10, weight='normal')
        
        # Mantener vista isométrica consistente
        ax1.view_init(elev=30, azim=45)
        ax1.grid(True, alpha=0.2)
        
        # Panel 2: Función de conectividad dinámica
        # Curva completa en gris claro (preview)
        ax2.plot(epc_results['radii'], epc_results['euler_chars'], 
                'lightgray', linewidth=1, alpha=0.3)
        
        # Curva hasta el punto actual
        ax2.plot(epc_results['radii'][:idx+1], epc_results['euler_chars'][:idx+1], 
                'b-', linewidth=3)
        
        # Punto actual grande y visible
        point_color = 'blue' if euler_char < 0 else 'red'
        ax2.plot(radius, euler_char, 'o', color=point_color, markersize=15, 
                zorder=5, markeredgecolor='darkred', markeredgewidth=2)
        
        # Línea de referencia χ = 0 (más prominente)
        ax2.axhline(y=0, color='red', linestyle='--', linewidth=2.5, alpha=0.8)
        ax2.text(epc_results['radii'][-1]*0.95, 0, 'χ = 0', 
                verticalalignment='bottom', horizontalalignment='right',
                fontsize=FONT_SIZE_SMALL, color='red',)
        
        # Radio crítico si existe
        if 'critical_radius' in epc_results and epc_results['critical_radius'] is not None:
            ax2.axvline(x=epc_results['critical_radius'], 
                       color='green', linestyle='--', linewidth=2.5,
                       label=f'$R_c$ ($\chi(r) = 0$) = {epc_results["critical_radius"]:.2f} μm')
        
        # Radio crítico físico si existe
        if 'critical_radius_physical' in epc_results and epc_results['critical_radius_physical'] is not None:
            ax2.axvline(x=epc_results['critical_radius_physical'], 
                       color='red', linestyle=':', linewidth=2.5,
                       label=f'$R_c$ (derivada) = {epc_results["critical_radius_physical"]:.2f} μm')
        
        # Área sombreada para resaltar regiones
        ax2.fill_between(epc_results['radii'][:idx+1], 0, epc_results['euler_chars'][:idx+1],
                        where=(epc_results['euler_chars'][:idx+1] < 0),
                        color='blue', alpha=0.1)
        ax2.fill_between(epc_results['radii'][:idx+1], 0, epc_results['euler_chars'][:idx+1],
                        where=(epc_results['euler_chars'][:idx+1] > 0),
                        color='red', alpha=0.1)
        
        # Configurar ejes con rango fijo
        ax2.set_xlim([0, epc_results['radii'][-1] * 1.05])
        ax2.set_ylim([euler_min-1, euler_max+1])
        ax2.set_xlabel('Radio de poro (μm)', fontsize=FONT_SIZE_MAIN)
        ax2.set_ylabel(r'$\chi(r)$ (adimensional)', fontsize=FONT_SIZE_MAIN)
        ax2.set_title(f'Evolución de Conectividad\nχ = {euler_char}', 
                     fontsize=FONT_SIZE_TITLE)
        ax2.grid(True, alpha=0.3, which='both')
        ax2.tick_params(axis='both', which='major', labelsize=FONT_SIZE_MAIN)
        
        # Añadir información adicional (con porosidad real)
        #info_text = (f'Radio: {radius:.3f} μm\n'
        #            f'$\phi$: {porosity_real * 100:.1f}%\n'
        #            f'Estado: {connectivity_state}')
        #ax2.text(0.02, 0.98, info_text, transform=ax2.transAxes, 
        #        fontsize=FONT_SIZE_SMALL, ha='left', va='top',
        #        bbox=dict(boxstyle='round,pad=0.5', facecolor='lightyellow', 
        #                 edgecolor='gray', alpha=0.9))
        
        # Leyenda si hay radio crítico
        if (('critical_radius' in epc_results and epc_results['critical_radius'] is not None) or
            ('critical_radius_physical' in epc_results and epc_results['critical_radius_physical'] is not None)):
            ax2.legend(loc='best', fontsize=FONT_SIZE_LEGEND, framealpha=0.9)
        
        # Título general con progreso e información del dominio
        progress = (frame_idx / (n_frames - 1)) * 100 if n_frames > 1 else 0
        domain_info = f" | Dominio: {domain_size_um[0]:.1f}×{domain_size_um[1]:.1f}×{domain_size_um[2]:.1f} μm³"
        fig.suptitle(f'EPC 3D Evolución {frame_idx+1}/{n_frames} | Progreso: {progress:.0f}%', 
                    fontsize=FONT_SIZE_SUPTITLE, y=0.98)
        
        # Ajustar layout
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    # Crear animación
    print("Generando frames de animación 3D...")
    anim = FuncAnimation(fig, update, frames=n_frames, interval=1000//fps, repeat=True)
    
    # Guardar como GIF
    filename = 'epc_evolution_3d_isometric.gif'
    print(f"Guardando animación como {filename}...")
    anim.save(filename, writer=PillowWriter(fps=fps))
    print(f"Animación EPC 3D isométrica guardada exitosamente")
    print(f"Dominio animado: {domain_size_um[0]:.1f} × {domain_size_um[1]:.1f} × {domain_size_um[2]:.1f} μm³")
    
    plt.close(fig)
    
    # Crear también una imagen estática comparativa si la función existe
    if 'create_epc_static_comparison' in globals():
        create_epc_static_comparison(epc_results)
    
    return anim

In [7]:
def main():
    """
    SUPER CÓDIGO 2000 CORREGIDO: Pipeline completo con cálculo correcto de EPC.
    """
    
    total_start = time.time()
    
    print("="*80)
    print("SUPER CÓDIGO 2000 CORREGIDO: ANÁLISIS COMPLETO CON EPC FIDEDIGNO")
    print("Versión corregida del cálculo de Euler-Poincaré")
    print("="*80)
    # PASO 1: Generar medio heterogéneo
    print("\nPASO 1: Generando medio poroso heterogéneo...")
    
    shape = (150, 150, 150)
    
    medium_info = create_heterogeneous_tight_rock(
        shape=shape,
        porosity_target=0.05,
        voxel_size_um=0.05,
        seed=42
    )
    
    # PASO 2: Análisis EPC mejorado con detección de cambios abruptos
    print("\nPASO 2: Análisis de conectividad EPC (versión corregida)...")
    
    epc_results = calculate_connectivity_function_enhanced(
        medium=medium_info['medium'],
        voxel_size_um=medium_info['voxel_size_um'],
        n_points=30
    )
    
    # PASO 3: Visualización mejorada  #####################################################################
    print("\nPASO 3: Visualizando evolución de conectividad con radios críticos...")
    
    try:
        fig_evolution = visualize_connectivity_evolution_enhanced(epc_results, medium_info['medium'])
    except Exception as e:
        print(f"Error en visualización EPC: {e}")

    

        
    # PASO 8: Animación EPC 3D    
    print("\nPASO 8: Creando animación EPC 3D isométrica...")
    
    try:
        anim_epc = create_animated_epc_evolution(epc_results, fps=2)
    except Exception as e:
        print(f"Error en animación EPC 3D: {e}")

    # RESUMEN FINAL mejorado
    total_time = time.time() - total_start
    
    print(f"\n{'='*80}")
    print(f"ANÁLISIS SUPER CÓDIGO 2000 CORREGIDO COMPLETADO")
    print(f"{'='*80}")
    print(f"Tiempo total: {total_time:.1f}s ({total_time/60:.1f} min)")
    
    print(f"\nArchivos generados:")
    print(f"• epc_connectivity_evolution_enhanced.png - Evolución con radios críticos")    

In [8]:
if __name__ == "__main__":
    # Ejecutar el análisis completo con EPC corregido
    main()

SUPER CÓDIGO 2000 CORREGIDO: ANÁLISIS COMPLETO CON EPC FIDEDIGNO
Versión corregida del cálculo de Euler-Poincaré

PASO 1: Generando medio poroso heterogéneo...
GENERANDO MEDIO POROSO HETEROGÉNEO OPTIMIZADO
Dimensiones: (150, 150, 150)
Porosidad objetivo: 5.0%
Tamaño de voxel: 0.05 μm

Distribución jerárquica de poros:
• Grandes (0.8 μm): 16 poros
• Medianos (0.4 μm): 101 poros
• Pequeños (0.2 μm): 506 poros

Creando red de poros con números de coordinación controlados...
  Generando 16 nodos de poros grandes...
  Generando 101 nodos de poros medianos...
  Generando 506 nodos de poros pequeños...

Conectando poros según números de coordinación...

Creando geometría de poros y gargantas...

Estadísticas de números de coordinación:
• Coordinación promedio: 2.76
• Rango: 1 - 6
Porosidad base de red: 0.04

Porosidad después de colocación: 0.04
Ajustando porosidad...

Verificando conectividad en los tres ejes:
• Conectividad eje X: 0.00
• Conectividad eje Y: 0.99
• Conectividad eje Z: 0.99



  Calculando EPC (método robusto corregido)...
    Componentes (β₀): 10
    Túneles/Loops (β₁): 0
    Cavidades (β₂): 0
    χ = β₀ - β₁ + β₂ = 10 - 0 + 0 = 10
    χ/V: 0.000322
    ✓ χ positiva indica baja conectividad (muchos componentes aislados)
  Calculando EPC (método robusto corregido)...
    Componentes (β₀): 10
    Túneles/Loops (β₁): 0
    Cavidades (β₂): 0
    χ = β₀ - β₁ + β₂ = 10 - 0 + 0 = 10
    χ/V: 0.000322
    ✓ χ positiva indica baja conectividad (muchos componentes aislados)
     0.39 |     10 |   10 |    0.0092 | 
  Calculando EPC (método robusto corregido)...
    Componentes (β₀): 7
    Túneles/Loops (β₁): 0
    Cavidades (β₂): 0
    χ = β₀ - β₁ + β₂ = 7 - 0 + 0 = 7
    χ/V: 0.000292
    ✓ χ positiva indica baja conectividad (muchos componentes aislados)
  Calculando EPC (método robusto corregido)...
    Componentes (β₀): 7
    Túneles/Loops (β₁): 0
    Cavidades (β₂): 0
    χ = β₀ - β₁ + β₂ = 7 - 0 + 0 = 7
    χ/V: 0.000292
    ✓ χ positiva indica baja conectividad